# HireMe.c
## Intro
While scrolling trough old [LiveOverflow](https://www.youtube.com/c/LiveOverflow) videos during dinner on Friday, I came across [this](https://www.youtube.com/watch?v=6sHSDoJ5a1s) video about [a programming challenge](https://www.nerd.nintendo.com/files/HireMe) by the Paris based Nintendo Europe R&D (NERD) department. Even though moving to France is not really an option, I was instantly hooked by this challenge, paused the video and got to work.

## Analyzing the Algorithm
The given C source file contains an algorithm that transforms a 32-byte input array into a 16-byte output array. In what follows, I will denote the input to the algorithm as $\vec{I}_0\in(\mathbb{Z}_2^{8})^{32}$. 

First, $\vec{I}_0$ is transformed 256-times in a loop. Let's denote the vector before the $i$-th loop iteration as $\vec{I}_{i-1}$, and the vector after it as $\vec{I}_i$. Now analyze a single step $\vec{I}_i\mapsto \vec{I}_{i+1}$.

Each step starts by applying a function $\sigma:\mathbb{Z}_2^8\rightarrow\mathbb{Z}_2^8$ component-wise, I denote this operation by $\sigma(\vec{I}_i)$. In the context of cryptography such functions are commonly called [S-boxes](https://en.wikipedia.org/wiki/S-box). The function is defined by the first 256 bytes of the `conf` array. As choosing good S-boxes is crucial for the security of a cryptographic algorithm we should at least briefly check that they are not completey flawed, e.g., whether they are affine, linear, or invertible.

Before executing this cell make sure to [load the requirements](#Requirements).

In [5]:
def f(x):
    assert(-1<x<256)
    return (conf[x]^conf[0])

assert(f(0b10)^f(0b01)!=f(0b10 ^ 0b01)) # not linear or affine
assert(f(24)==f(94)) # not invertible

Next, the step proceeds by applying a *linear* map $C:(\mathbb{Z}_2^{8})^{32}\rightarrow (\mathbb{Z}_2^{8})^{32}$. The coefficients of this matrix are given by the bits of the 32 32-bit integers in the `diff` array. Each integer defines a row and I'll use the convention that the least significant bit is in the first column. You may wonder how to apply a $32x32$ matrix to a $32*8$ element vector, the simple answer is: replace $1\mapsto\mathbb{1}_8$ and $0\mapsto 0_8$ to get a block matrix of the correct size.

Aside: One could equivalently phrase all this math in term of the finite field $GF(2^8)$ and work with $32x32$ matrices and $32$-element vectors. Our approach is equivalent to that since $GF(2^8)\cong GF(2)^8$ (as vector spaces over the prime subfield) and multiplication by a field element defines a linear map, i.e., an $8x8$ matrix, on this vector space.

So far we have that
$$
\vec{I}_i=C\sigma\vec{I}_{i-1}\quad\text{ and thus in particular }\quad \vec{I}_{256}=(C\sigma)^{256}\vec{I}_0.
$$

The last step is to project the result $\vec{I}_{256}$ down to $(\mathbb{Z}_2^{8})^{16}$ to obtain the 16-byte output vector. However, prior to that $\vec{I}_{256}$ is fed through a component-wise *alternating* S-box, i.e., we apply $\sigma$ to components with even index and $\tau$ to components with odd index, where $\tau:\mathbb{Z}_2^8\rightarrow\mathbb{Z}_2^8$ is defined by the last 256 bytes of the `conf` array. I'll indicate the component-wise application by a vector of maps
$$
\vec{I}_{256}\mapsto\begin{pmatrix}\begin{align}\sigma \\ \tau \\ \sigma \\ \tau \\ \vdots \end{align}\end{pmatrix}\vec{I}_{256}\,.
$$

Finally, we apply a projection $P:(\mathbb{Z}_2^8)^{32}\rightarrow (\mathbb{Z}_2^8)^{16}$ to get the 16-byte output value $\vec{O}\in(\mathbb{Z}_2^8)^{16}$. The structure of the matrix is as follows
$$
P=\begin{pmatrix}
\begin{align}
 \mathbb{1}_8 & \mathbb{1}_8 & 0_8 & \dots & \dots & \dots & \dots & \newline 
 0_8 & 0_8 & \mathbb{1}_8 & \mathbb{1}_8 & 0_8 & \dots & \dots &  \newline
 \vdots & \dots & \dots & 0_8 & \mathbb{1}_8 & \mathbb{1}_8 & 0_8 & \dots \newline
 \vdots
\end{align}
\end{pmatrix},
$$
i.e., it halves the dimension of its input vector by combining neighbouring bytes with an xor operation. We can summarize our analysis by saying that the algorithm implements the mapping
$$
\vec{I}_{0}\mapsto \vec{O}[\vec{I}_{0}]=P\begin{pmatrix}\begin{align}\sigma \\ \tau \\ \sigma \\ \tau \\ \vdots \end{align}\end{pmatrix}(C\sigma)^{256}\vec{I}_{0}\,.
$$

## Reversing the Algorithm
Let's assume that we want to generate inputs to the algorithm that produce some particular output $\vec{O}\in(\mathbb{Z}_2^8)^{16}$. The pre-image of $\vec{O}$ under the projection is of course highly non-unique, in fact, there are $2^{8*16}$ possible $\vec{v}\in(\mathbb{Z}_2^8)^{32}$ such that $\vec{O}=P\vec{v}$. However, note that since $\sigma$ and $\tau$ are non-invertible some of those pre-images may be unreachable for the alternating component-wise S-box. Efficiently generating pre-images could be done by choosing some enumeration of the Cartesian product $\mathrm{Im}(\sigma)^{16}$ (for choosing the even components) and then picking the odd components such that the xor of neighbours yields us $\vec{O}$ (of course we must make sure that all odd components are in $\mathrm{Im}(\tau)$).

For a given pre-image, inverting the alternating S-box might produce multiple pre-pre-images, and we should pursue each of them further.

Now to the ostensibly most scary part: the "256-round gigantic xor mixer". We have already inverted the $\sigma$ S-box so this isn't a problem anymore. Now, there're essentially 2 possiblilieties: either $C$ is invertible, or it is not. Let's hope for the former as it would save us some pain. Recall that a square matrix is of full rank iff it is invertible. Asking Google + [Stackoverflow](https://stackoverflow.com/questions/56856378/fast-computation-of-matrix-rank-over-gf2) for an algorithm to find the rank of a matrix over $GF(2)$ gives us an answer in a matter of seconds:

In [6]:
def gf2_rank(rows):
    """
    Find rank of a matrix over GF2.
    
    https://stackoverflow.com/questions/56856378/fast-computation-of-matrix-rank-over-gf2
    
    The rows of the matrix are given as nonnegative integers, thought
    of as bit-strings.

    This function modifies the input list. Use gf2_rank(rows.copy())
    instead of gf2_rank(rows) to avoid modifying rows.
    """
    rank = 0
    while rows:
        pivot_row = rows.pop()
        if pivot_row:
            rank += 1
            lsb = pivot_row & -pivot_row
            for index, row in enumerate(rows):
                if row & lsb:
                    rows[index] = row ^ pivot_row
    return rank

assert(gf2_rank(diff.copy())==32) # we can work with a 32x32 matrix due to block form

Yay, jackpot, so dear Sir. Google + Stackoverflow, please give me an algorithm for the inverse of this matrix ... okay, okay I'll do it myself, [Gauss-Jordan elimination](https://en.wikipedia.org/wiki/Gaussian_elimination#Finding_the_inverse_of_a_matrix) ain't that hard and we can be lazy since it must only work over $GF(2)$ and we can assume that the matrix is invertible. Here's some code that does the trick, its a bit hacky so don't cite me on this one :)

In [7]:
def gf2_invert(rows: list[int], n=32) -> list[int]:
    '''
    Calculate the inverse of a nxn matrix over GF(2) with Gauss-Jordan
    elimination.
    '''
    rowsI = [(rows[i] << n) + (1 << (n-1-i)) for i in range(n)]
    for i in range(n):
        rowsI.sort(reverse=True)
        pivot_row = rowsI[i]
        msb, _ = _msb(pivot_row)
        for j in range(n):
            if i == j:
                continue
            if msb & rowsI[j]:
                rowsI[j] ^= pivot_row
    return [i & (2**n - 1) for i in rowsI]

 
def gf2_mat_mul(rowsA: list[int], rowsB: list[int], n=32) -> list[int]:
    '''
    Calculates matrix product.
    '''
    rowsAB = [0 for _ in range(n)]
    for row, rowA in enumerate(rowsA):
        for column in range(n):
            rowsAB[row] |= _gf2_sum(_gf2_get_column(rowsB, column, n) & rowA) << (n-1-column)
    return rowsAB

def gf2_mat_print(rows: list[int], n=32) -> None:
    '''
    Pretty printer.
    '''
    for row in rows:
        print(f"{row:0{n}b}")
        
C = [_reverse_int(i) for i in diff]
gf2_mat_print(gf2_mat_mul(C, gf2_invert(C))) # CC^-1 = id

10000000000000000000000000000000
01000000000000000000000000000000
00100000000000000000000000000000
00010000000000000000000000000000
00001000000000000000000000000000
00000100000000000000000000000000
00000010000000000000000000000000
00000001000000000000000000000000
00000000100000000000000000000000
00000000010000000000000000000000
00000000001000000000000000000000
00000000000100000000000000000000
00000000000010000000000000000000
00000000000001000000000000000000
00000000000000100000000000000000
00000000000000010000000000000000
00000000000000001000000000000000
00000000000000000100000000000000
00000000000000000010000000000000
00000000000000000001000000000000
00000000000000000000100000000000
00000000000000000000010000000000
00000000000000000000001000000000
00000000000000000000000100000000
00000000000000000000000010000000
00000000000000000000000001000000
00000000000000000000000000100000
00000000000000000000000000010000
00000000000000000000000000001000
00000000000000000000000000000100
0000000000

When analyzing an algorithm in a CTF challenge its always a good idea to play arround with it's constituent pieces a bit before heading off into a certain direction. Those custom crypto implementation are usually messed up and seeing the output for different inputs or the evolution of internal state along the way can help to spot flaws that save you a lot of time. If I would have done that, I'd maybe have noticed the following earlier, which would have saved me some time indeed ...

In [8]:
assert(gf2_invert(C)==C) # C is an involutory matrix!

When reversing any one of the 256 $(C\sigma)$-iterations, applying $C$ might lead to a vector with an component outside of $\mathrm{Im}(\sigma)$, those need to be discarded. Furthermore, inverting the S-box may produce multiple possible preimages, each of which needs to be tracked further, but that shouldn't be a problem either. In theory, we now have a plan for reverseing the algorithm and finding a job-winning input in finite time.

## Implementation Steps
Disclaimer: This is just my approach, I know there're better ones, but when working on such a challenge it's always a tradeoff between writing the "most beautiful and utterly perfect" algorithm and just getting stuff done. Feel free to let me know about your ideas in the comments!  

My goal was to write an algorithm capable of finding **all** inputs that produce the required output, i.e., complete NERD's *Level 3*. In this section I descibe the algorithm in prose, [jump down](#Implementation_and_Performance) for the commented source code.

First, there is some initialization work to do. The `main`-function starts by building a lookup table for inverting the S-boxes (`SInv` and `TInv`) and also initializes hash-maps to efficiently check if an element is in the image (`ImT` and `ImS_set`).

Next, we need a way to enumerate all preimages $\vec{v}\in(\mathbb{Z}_2^8)^{32}$ such that $P\vec{v}=\vec{O}$ as each of them needs to be traced backwards. What is more, $\vec{v}$ needs to be in the image of the alternating S-box. As hinted at above, I decided to enumerate the Cartesian product $\mathrm{Im}(\sigma)^{16}$ to get all possible combinations of even components. Then I simply iterate over them, check if the odd components we need are in $\mathrm{Im}(\tau)$, and start tracing the candidate backwards through the algorithm. Since the product is $O(2^{128})$ enumerating isn't quite as easy as `itertools.product`, so I settled for using a positive integer counter's base-$|\mathrm{Im}(\sigma)|$ coefficients. This approach is implemented in `candidate_nr_to_candidate`, which is probably quite computationally expensive due to the `log` operations and divisions by larger powers of the base. Inefficientcy aside, given an index this function returns a preimage or, if the preimage isn't in the image of the alternating S-box, an error code. 

Aside: When starting at a bad position in the search space this approach could mean counting till the end of time without ever reaching an invertible pre-image, effectively being stuck at this step forever. This is due to factors in the Cartesian product that correspond to high coefficients of `candidate_nr` being effectively constant, meaning we are lost if the needed odd neighbour of them is not in $\mathrm{Im}(\tau)$. However, here it isn't a problem and in general there are ways to fix this.

The rest is essentially a `while True` loop that starts by picking a `candiate` and inverting the alternating S-box to obtain a list of `preimages`. For every `preimage` it iterates over all `_input` that could have produced it. I decided to obtain those `_input`s by a recursive depth-first-search (DFS) for simplicity. Each recursion starts by undoing $C$ and then proceeds with inverting the S-box, aborting this path if inverting is not possible. It then recursively calls itself for each preimage found. While this is appealingly short, other approaches may be much faster, however, remember we also want to get stuff done ;)

<a id='Implementation_and_Performance'></a>
## Implementation and Performance
Below is my Python implementation of the steps described above. I'm really bad at writing fast Python code so its probably full of gotchas, feel free to point them out in the comments! However, it still manages to find plenty of valid inputs in under a minute on my 7-year-old laptop.

In [9]:
def main(_print=True, want=1) -> int:
    '''
    _print: bool: print new inputs as soon as we find them
    want:   int:  min. number of inputs to generate
    Returns: actual nr of inputs found
    '''
    global conf, diff
    # Desired outcome ;)
    solution = b"Hire me!!!!!!!!\x00"
    # How many inputs we have generated so far
    have = 0

    # Initialization
    #   Image and pre-image of S-boxes
    SInv = [set() for _ in range(256)]
    TInv = [set() for _ in range(256)]
    ImS = []
    ImT = set()
    for i in range(256):
        SInv[conf[i]].update({i})
        if conf[i] not in ImS:
            ImS.append(conf[i])
        TInv[conf[i+256]].update({i})
        ImT.update({conf[i+256]})
    ImS_set = set(ImS)
    #   Inverse of matrix C
    C = (_reverse_int(i) for i in diff)
    C = tuple(C)
    #   Matrix vector mult for id block matrix
    def gf2_mat_vec_mul(rows: tuple, vec: tuple, n=32) -> tuple:
        res = [0 for _ in range(n)]
        for i, row in enumerate(rows):
            for j, x in enumerate(vec):
                res[i] ^= x * ((row >> (n-1-j)) & 1)
        return tuple(res)
    
    # Counter interpreted in base-|Im(S)| to enumerate the cartesian product ImS^x16
    candidate_nr = 0
    candidate = [0 for _ in range(32)]
    def candidate_nr_to_candidate(nr: int):
        nonlocal ImS, ImT, solution
        base = len(ImS)
        err = 0
        
        candidate = [0 for _ in range(32)]
        coefficients = [0 for _ in range(16)]
        
        while nr > 0: # find coefficients in base-|Im(S)|
            max_idx = math.floor(math.log(nr, base))
            coefficients[max_idx] = math.floor(nr/(base**max_idx))
            nr -= coefficients[max_idx] * base**max_idx
        
        for idx, coefficient in enumerate(coefficients):
            cImS = ImS[coefficient]
            cImT = cImS ^ solution[idx] # required neighbour to get solution
            if cImT not in ImT: # required neighbour is not in Im(T)
                err = 1
                return (candidate, err)
            candidate[2*idx] = cImS
            candidate[2*idx+1] = cImT
        
        return (candidate, err)
       
    # A simple recursive DFS to trace a candidate c backwards through (CS)^n
    # Returns list of all inputs I s.t. c=(CS)^n I (emtpy list if there are none).
    def undo_n_C_S(_input: tuple, n):
        nonlocal SInv, ImS_set, C
        if n == 0:
            if _print:
                print(_input)
            return _input
        ret = []
        next_inputs = [0 for _ in range(32)]
        preimage = gf2_mat_vec_mul(C, _input)
        for i, x in enumerate(preimage):
            if x not in ImS_set: # this preimage cannot be reached
                return ret
            next_inputs[i] = SInv[x]
        for next_input in itertools.product(*next_inputs):
            tmp = undo_n_C_S(next_input, n-1)
            if tmp:
                if type(tmp)==tuple:
                    ret.append(tmp)
                else:
                    for t in tmp:
                        ret.append(t)
        return ret    
    
    # Main loop
    while have < want:
        # convert candidate number to element of cartesian product, finds summands in ImT 
        candidate, err = candidate_nr_to_candidate(candidate_nr)
        # bump candidate number 
        candidate_nr += 1
        if err:
            # at least one required summand was not in ImT, skip
            continue
        # Invert alternating S-box
        preimages = [TInv[cIm] if (idx & 1) else SInv[cIm] for idx, cIm in enumerate(candidate)]
        # Loop over all possible pre-images of candiate
        for preimage in itertools.product(*preimages):
            for _input in undo_n_C_S(preimage, 256):
                have += 1
    
    return have
                        
print("Total nr of inputs generated: ", main())

(49, 183, 244, 72, 53, 143, 214, 87, 0, 24, 125, 12, 197, 43, 15, 231, 244, 179, 223, 169, 247, 153, 41, 177, 222, 167, 221, 184, 91, 38, 29, 22)
(49, 183, 244, 72, 53, 143, 214, 87, 0, 24, 125, 12, 197, 43, 15, 231, 244, 179, 223, 169, 247, 153, 41, 177, 222, 167, 221, 207, 91, 38, 29, 22)
(49, 183, 244, 72, 53, 143, 214, 87, 0, 24, 125, 12, 197, 43, 15, 231, 244, 179, 223, 169, 247, 153, 181, 177, 222, 167, 221, 184, 91, 38, 29, 22)
(49, 183, 244, 72, 53, 143, 214, 87, 0, 24, 125, 12, 197, 43, 15, 231, 244, 179, 223, 169, 247, 153, 181, 177, 222, 167, 221, 207, 91, 38, 29, 22)
(49, 183, 244, 72, 53, 143, 214, 87, 0, 94, 125, 12, 197, 43, 15, 231, 244, 179, 223, 169, 247, 153, 41, 177, 222, 167, 221, 184, 91, 38, 29, 22)
(49, 183, 244, 72, 53, 143, 214, 87, 0, 94, 125, 12, 197, 43, 15, 231, 244, 179, 223, 169, 247, 153, 41, 177, 222, 167, 221, 207, 91, 38, 29, 22)
(49, 183, 244, 72, 53, 143, 214, 87, 0, 94, 125, 12, 197, 43, 15, 231, 244, 179, 223, 169, 247, 153, 181, 177, 222, 167, 2

(17, 128, 158, 145, 23, 9, 249, 12, 174, 221, 236, 42, 115, 241, 5, 105, 178, 44, 155, 148, 232, 114, 19, 72, 217, 134, 91, 216, 160, 237, 4, 227)
(17, 128, 158, 145, 23, 9, 249, 12, 174, 221, 236, 42, 115, 241, 5, 105, 178, 44, 155, 148, 232, 114, 19, 72, 217, 134, 91, 216, 160, 111, 4, 227)
(17, 128, 158, 53, 180, 133, 249, 12, 174, 87, 147, 42, 208, 49, 251, 105, 230, 44, 169, 148, 232, 123, 19, 228, 217, 134, 78, 242, 192, 237, 4, 227)
(17, 128, 158, 53, 180, 133, 249, 12, 174, 87, 147, 42, 208, 49, 251, 105, 230, 44, 169, 148, 232, 123, 19, 228, 217, 134, 78, 242, 192, 111, 4, 227)
(48, 159, 200, 143, 144, 66, 174, 197, 8, 37, 169, 30, 180, 163, 201, 137, 220, 27, 46, 236, 78, 48, 82, 104, 192, 2, 59, 163, 79, 84, 115, 174)
(48, 159, 200, 143, 144, 66, 174, 197, 8, 37, 169, 30, 180, 163, 201, 137, 132, 27, 46, 236, 78, 48, 82, 104, 192, 2, 59, 163, 79, 84, 115, 174)
(48, 159, 200, 143, 144, 66, 174, 197, 8, 37, 169, 30, 180, 163, 201, 171, 220, 27, 46, 236, 78, 48, 82, 104, 192, 2

(9, 193, 153, 192, 202, 4, 142, 163, 184, 109, 82, 52, 68, 253, 12, 71, 61, 130, 191, 212, 149, 211, 143, 140, 188, 93, 28, 167, 50, 49, 89, 159)
(9, 193, 153, 192, 202, 4, 142, 163, 207, 109, 82, 52, 68, 253, 12, 71, 61, 130, 191, 212, 149, 211, 143, 140, 188, 93, 28, 167, 50, 49, 89, 159)
(206, 116, 153, 192, 240, 4, 67, 128, 184, 109, 87, 129, 29, 84, 12, 248, 61, 130, 191, 148, 122, 211, 4, 140, 188, 123, 28, 167, 50, 49, 105, 159)
(206, 116, 153, 192, 240, 4, 67, 128, 207, 109, 87, 129, 29, 84, 12, 248, 61, 130, 191, 148, 122, 211, 4, 140, 188, 123, 28, 167, 50, 49, 105, 159)
(95, 193, 210, 190, 14, 218, 142, 177, 124, 109, 102, 212, 68, 93, 19, 88, 61, 178, 153, 52, 235, 37, 196, 140, 188, 219, 28, 40, 246, 235, 89, 159)
(95, 193, 210, 190, 14, 218, 142, 177, 36, 109, 102, 212, 68, 93, 19, 88, 61, 178, 153, 52, 235, 37, 196, 140, 188, 219, 28, 40, 246, 235, 89, 159)
(38, 116, 210, 190, 187, 218, 67, 140, 124, 109, 195, 148, 29, 123, 19, 231, 61, 178, 153, 129, 26, 37, 53, 140, 18

(117, 68, 39, 172, 98, 245, 216, 75, 45, 215, 249, 119, 18, 98, 228, 252, 154, 231, 64, 164, 206, 237, 190, 147, 47, 148, 35, 236, 19, 69, 242, 205)
(117, 68, 39, 172, 98, 245, 216, 75, 45, 215, 249, 119, 18, 98, 228, 252, 154, 231, 64, 164, 206, 111, 190, 147, 47, 148, 35, 236, 19, 69, 242, 205)
(117, 68, 39, 172, 98, 245, 216, 75, 45, 215, 249, 119, 18, 98, 228, 252, 154, 231, 55, 164, 206, 237, 190, 147, 47, 148, 35, 236, 19, 69, 242, 205)
(117, 68, 39, 172, 98, 245, 216, 75, 45, 215, 249, 119, 18, 98, 228, 252, 154, 231, 55, 164, 206, 111, 190, 147, 47, 148, 35, 236, 19, 69, 242, 205)
(117, 68, 39, 172, 98, 245, 216, 75, 45, 215, 249, 119, 18, 98, 228, 252, 6, 231, 64, 164, 206, 237, 190, 147, 47, 148, 35, 236, 19, 69, 242, 205)
(117, 68, 39, 172, 98, 245, 216, 75, 45, 215, 249, 119, 18, 98, 228, 252, 6, 231, 64, 164, 206, 111, 190, 147, 47, 148, 35, 236, 19, 69, 242, 205)
(117, 68, 39, 172, 98, 245, 216, 75, 45, 215, 249, 119, 18, 98, 228, 252, 6, 231, 55, 164, 206, 237, 190, 147,

(60, 255, 153, 169, 229, 84, 178, 66, 75, 44, 24, 98, 98, 175, 90, 240, 66, 194, 200, 200, 53, 138, 18, 71, 118, 172, 75, 39, 208, 59, 232, 205)
(60, 255, 153, 169, 229, 84, 178, 66, 75, 44, 24, 98, 98, 175, 90, 240, 66, 11, 200, 200, 53, 138, 18, 71, 118, 172, 75, 39, 208, 59, 232, 205)
(60, 255, 153, 169, 229, 84, 178, 66, 75, 44, 24, 98, 254, 175, 90, 240, 66, 194, 200, 200, 53, 138, 18, 71, 118, 172, 75, 39, 208, 59, 232, 205)
(60, 255, 153, 169, 229, 84, 178, 66, 75, 44, 24, 98, 254, 175, 90, 240, 66, 11, 200, 200, 53, 138, 18, 71, 118, 172, 75, 39, 208, 59, 232, 205)
(60, 255, 153, 169, 229, 84, 178, 66, 75, 44, 24, 254, 98, 175, 90, 240, 66, 194, 200, 200, 53, 138, 18, 71, 118, 172, 75, 39, 208, 59, 232, 205)
(60, 255, 153, 169, 229, 84, 178, 66, 75, 44, 24, 254, 98, 175, 90, 240, 66, 11, 200, 200, 53, 138, 18, 71, 118, 172, 75, 39, 208, 59, 232, 205)
(60, 255, 153, 169, 229, 84, 178, 66, 75, 44, 24, 254, 254, 175, 90, 240, 66, 194, 200, 200, 53, 138, 18, 71, 118, 172, 75, 39, 2

...

(205, 188, 86, 155, 239, 228, 157, 167, 98, 140, 45, 129, 29, 203, 231, 157, 103, 238, 25, 190, 125, 162, 135, 167, 149, 124, 125, 221, 203, 214, 12, 220)
(205, 188, 86, 155, 239, 228, 157, 167, 98, 140, 45, 129, 29, 203, 231, 157, 103, 238, 25, 190, 125, 162, 135, 167, 149, 124, 125, 221, 203, 214, 12, 132)
(205, 188, 86, 155, 239, 228, 157, 167, 98, 140, 45, 129, 29, 203, 231, 157, 103, 238, 25, 190, 125, 162, 135, 167, 149, 36, 125, 221, 203, 214, 12, 220)
(205, 188, 86, 155, 239, 228, 157, 167, 98, 140, 45, 129, 29, 203, 231, 157, 103, 238, 25, 190, 125, 162, 135, 167, 149, 36, 125, 221, 203, 214, 12, 132)
(205, 188, 86, 155, 239, 228, 157, 167, 254, 140, 45, 129, 29, 203, 231, 157, 103, 238, 25, 190, 125, 162, 135, 167, 149, 124, 125, 221, 203, 214, 12, 220)
(205, 188, 86, 155, 239, 228, 157, 167, 254, 140, 45, 129, 29, 203, 231, 157, 103, 238, 25, 190, 125, 162, 135, 167, 149, 124, 125, 221, 203, 214, 12, 132)
(205, 188, 86, 155, 239, 228, 157, 167, 254, 140, 45, 129, 29, 203, 23

(81, 233, 64, 227, 59, 108, 190, 64, 23, 72, 154, 67, 131, 121, 232, 2, 113, 169, 63, 229, 136, 70, 69, 225, 69, 143, 167, 250, 67, 76, 45, 102)
(81, 233, 64, 227, 59, 108, 190, 64, 23, 72, 154, 67, 131, 121, 232, 2, 83, 169, 63, 229, 136, 70, 69, 225, 69, 143, 167, 250, 67, 76, 45, 102)
(81, 233, 64, 227, 59, 108, 190, 64, 23, 72, 6, 67, 131, 121, 232, 2, 113, 169, 63, 229, 136, 70, 69, 225, 69, 143, 167, 250, 67, 76, 45, 102)
(81, 233, 64, 227, 59, 108, 190, 64, 23, 72, 6, 67, 131, 121, 232, 2, 83, 169, 63, 229, 136, 70, 69, 225, 69, 143, 167, 250, 67, 76, 45, 102)
(81, 233, 64, 227, 59, 108, 190, 55, 23, 72, 154, 67, 131, 121, 232, 2, 113, 169, 63, 229, 136, 70, 69, 225, 69, 143, 167, 250, 67, 76, 45, 102)
(81, 233, 64, 227, 59, 108, 190, 55, 23, 72, 154, 67, 131, 121, 232, 2, 83, 169, 63, 229, 136, 70, 69, 225, 69, 143, 167, 250, 67, 76, 45, 102)
(81, 233, 64, 227, 59, 108, 190, 55, 23, 72, 6, 67, 131, 121, 232, 2, 113, 169, 63, 229, 136, 70, 69, 225, 69, 143, 167, 250, 67, 76, 45,

(201, 78, 8, 41, 60, 236, 251, 82, 238, 97, 86, 18, 188, 57, 112, 3, 57, 249, 122, 241, 19, 164, 42, 153, 154, 58, 187, 234, 1, 2, 239, 246)
(201, 78, 8, 41, 60, 236, 251, 82, 238, 97, 86, 18, 188, 57, 112, 3, 57, 249, 122, 241, 19, 164, 42, 153, 154, 243, 187, 234, 1, 2, 239, 246)
(201, 78, 8, 41, 60, 236, 251, 82, 238, 97, 86, 18, 188, 57, 112, 3, 57, 249, 122, 241, 19, 164, 42, 153, 6, 58, 187, 234, 1, 2, 239, 246)
(201, 78, 8, 41, 60, 236, 251, 82, 238, 97, 86, 18, 188, 57, 112, 3, 57, 249, 122, 241, 19, 164, 42, 153, 6, 243, 187, 234, 1, 2, 239, 246)
(201, 78, 8, 181, 60, 236, 251, 82, 238, 97, 86, 18, 188, 57, 112, 3, 57, 249, 122, 241, 19, 164, 42, 153, 154, 58, 187, 234, 1, 2, 239, 246)
(201, 78, 8, 181, 60, 236, 251, 82, 238, 97, 86, 18, 188, 57, 112, 3, 57, 249, 122, 241, 19, 164, 42, 153, 154, 243, 187, 234, 1, 2, 239, 246)
(201, 78, 8, 181, 60, 236, 251, 82, 238, 97, 86, 18, 188, 57, 112, 3, 57, 249, 122, 241, 19, 164, 42, 153, 6, 58, 187, 234, 1, 2, 239, 246)
(201, 78, 8, 

(105, 246, 110, 200, 96, 13, 238, 135, 19, 223, 189, 246, 15, 45, 40, 148, 148, 46, 102, 189, 216, 19, 200, 64, 2, 78, 9, 119, 237, 15, 219, 212)
(105, 246, 110, 200, 96, 13, 238, 135, 19, 223, 189, 246, 15, 45, 40, 148, 148, 46, 102, 189, 216, 19, 200, 64, 2, 78, 9, 119, 111, 15, 219, 212)
(105, 246, 110, 200, 96, 13, 238, 135, 19, 223, 189, 246, 15, 45, 40, 148, 148, 46, 102, 189, 216, 19, 200, 55, 2, 78, 9, 119, 237, 15, 219, 212)
(105, 246, 110, 200, 96, 13, 238, 135, 19, 223, 189, 246, 15, 45, 40, 148, 148, 46, 102, 189, 216, 19, 200, 55, 2, 78, 9, 119, 111, 15, 219, 212)
(105, 250, 110, 200, 245, 13, 238, 135, 18, 116, 189, 250, 15, 45, 147, 148, 148, 46, 82, 203, 216, 18, 173, 64, 242, 174, 208, 35, 237, 15, 219, 80)
(105, 250, 110, 200, 245, 13, 238, 135, 18, 116, 189, 250, 15, 45, 147, 148, 148, 46, 82, 203, 216, 18, 173, 64, 242, 174, 208, 35, 111, 15, 219, 80)
(105, 250, 110, 200, 245, 13, 238, 135, 18, 116, 189, 250, 15, 45, 147, 148, 148, 46, 82, 203, 216, 18, 173, 55, 242

Total nr of inputs generated:  4512


We can use [`cProfile`](https://docs.python.org/3/library/profile.html#instant-user-s-manual) to create a simple profile of the algorithm. 

In [10]:
cProfile.run('main(_print=False)')

         4325689 function calls (3533418 primitive calls) in 246.759 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.006    0.006  246.759  246.759 1752670724.py:1(main)
        1    0.000    0.000    0.000    0.000 1752670724.py:15(<listcomp>)
        1    0.000    0.000    0.000    0.000 1752670724.py:16(<listcomp>)
       33    0.000    0.000    0.000    0.000 1752670724.py:27(<genexpr>)
   788793  241.122    0.000  242.069    0.000 1752670724.py:30(gf2_mat_vec_mul)
   788793    0.946    0.000    0.946    0.000 1752670724.py:31(<listcomp>)
        1    0.000    0.000    0.000    0.000 1752670724.py:39(<listcomp>)
      968    0.010    0.000    0.013    0.000 1752670724.py:40(candidate_nr_to_candidate)
      968    0.001    0.000    0.001    0.000 1752670724.py:45(<listcomp>)
      968    0.001    0.000    0.001    0.000 1752670724.py:46(<listcomp>)
793305/1034    3.588    0.000  246.735    0.239 1752670724

On my machine, `main` takes about 240 seconds to generate 4512 inputs, however, all but 0.005s of this time are spent in function calls. Essentially, most of its time the program is multiplying matrices in calls to `gf2_mat_vec_mul` inside `undo_n_C_S`. I thought a for bit about ways to optimize aroud this observation but didn't get any good ideas. To me, the function itself looks pretty efficient (for pure python). One could try to reduce calls to it but I don't see an obvious way to do that.

That's it, weekend's over, took me Friday night and the best part of Saturday to figure out the solution, then Sunday afternoon to write it all up in this Notebook - now it's time to climb out of this rabbit hole and get back to the real work.

<a id='Requirements'></a>
## Requirements
Execute these cells first, I moved them here so they don't clutter the notebook. The goal was to write everything in pure Python so there's no need to install any packages.

### Imports

In [2]:
import math
import itertools
import cProfile

### Constants from challenge

In [3]:
conf=[0xac,0xd1,0x25,0x94,0x1f,0xb3,0x33,0x28,0x7c,0x2b,0x17,0xbc,0xf6,0xb0,0x55,0x5d,
0x8f,0xd2,0x48,0xd4,0xd3,0x78,0x62,0x1a,0x02,0xf2,0x01,0xc9,0xaa,0xf0,0x83,0x71,
0x72,0x4b,0x6a,0xe8,0xe9,0x42,0xc0,0x53,0x63,0x66,0x13,0x4a,0xc1,0x85,0xcf,0x0c,
0x24,0x76,0xa5,0x6e,0xd7,0xa1,0xec,0xc6,0x04,0xc2,0xa2,0x5c,0x81,0x92,0x6c,0xda,
0xc6,0x86,0xba,0x4d,0x39,0xa0,0x0e,0x8c,0x8a,0xd0,0xfe,0x59,0x96,0x49,0xe6,0xea,
0x69,0x30,0x52,0x1c,0xe0,0xb2,0x05,0x9b,0x10,0x03,0xa8,0x64,0x51,0x97,0x02,0x09,
0x8e,0xad,0xf7,0x36,0x47,0xab,0xce,0x7f,0x56,0xca,0x00,0xe3,0xed,0xf1,0x38,0xd8,
0x26,0x1c,0xdc,0x35,0x91,0x43,0x2c,0x74,0xb4,0x61,0x9d,0x5e,0xe9,0x4c,0xbf,0x77,
0x16,0x1e,0x21,0x1d,0x2d,0xa9,0x95,0xb8,0xc3,0x8d,0xf8,0xdb,0x34,0xe1,0x84,0xd6,
0x0b,0x23,0x4e,0xff,0x3c,0x54,0xa7,0x78,0xa4,0x89,0x33,0x6d,0xfb,0x79,0x27,0xc4,
0xf9,0x40,0x41,0xdf,0xc5,0x82,0x93,0xdd,0xa6,0xef,0xcd,0x8d,0xa3,0xae,0x7a,0xb6,
0x2f,0xfd,0xbd,0xe5,0x98,0x66,0xf3,0x4f,0x57,0x88,0x90,0x9c,0x0a,0x50,0xe7,0x15,
0x7b,0x58,0xbc,0x07,0x68,0x3a,0x5f,0xee,0x32,0x9f,0xeb,0xcc,0x18,0x8b,0xe2,0x57,
0xb7,0x49,0x37,0xde,0xf5,0x99,0x67,0x5b,0x3b,0xbb,0x3d,0xb5,0x2d,0x19,0x2e,0x0d,
0x93,0xfc,0x7e,0x06,0x08,0xbe,0x3f,0xd9,0x2a,0x70,0x9a,0xc8,0x7d,0xd8,0x46,0x65,
0x22,0xf4,0xb9,0xa2,0x6f,0x12,0x1b,0x14,0x45,0xc7,0x87,0x31,0x60,0x29,0xf7,0x73,
0x2c,0x97,0x72,0xcd,0x89,0xa6,0x88,0x4c,0xe8,0x83,0xeb,0x59,0xca,0x50,0x3f,0x27,
0x4e,0xae,0x43,0xd5,0x6e,0xd0,0x99,0x7b,0x7c,0x40,0x0c,0x52,0x86,0xc1,0x46,0x12,
0x5a,0x28,0xa8,0xbb,0xcb,0xf0,0x11,0x95,0x26,0x0d,0x34,0x66,0x22,0x18,0x6f,0x51,
0x9b,0x3b,0xda,0xec,0x5e,0x00,0x2a,0xf5,0x8f,0x61,0xba,0x96,0xb3,0xd1,0x30,0xdc,
0x33,0x75,0xe9,0x6d,0xc8,0xa1,0x3a,0x3e,0x5f,0x9d,0xfd,0xa9,0x31,0x9f,0xaa,0x85,
0x2f,0x92,0xaf,0x67,0x78,0xa5,0xab,0x03,0x21,0x4f,0xb9,0xad,0xfe,0xf3,0x42,0xfc,
0x17,0xd7,0xee,0xa3,0xd8,0x80,0x14,0x2e,0xa0,0x47,0x55,0xc4,0xff,0xe5,0x13,0x3f,
0x81,0xb6,0x7a,0x94,0xd0,0xb5,0x54,0xbf,0x91,0xa7,0x37,0xf1,0x6b,0xc9,0x1b,0xb1,
0x3c,0xb6,0xd9,0x32,0x24,0x8d,0xf2,0x82,0xb4,0xf9,0xdb,0x7d,0x44,0xfb,0x1e,0xd4,
0xea,0x5d,0x35,0x69,0x23,0x71,0x57,0x01,0x06,0xe4,0x55,0x9a,0xa4,0x58,0x56,0xc7,
0x4a,0x8c,0x8a,0xd6,0x6a,0x49,0x70,0xc5,0x8e,0x0a,0x62,0xdc,0x29,0x4b,0x42,0x41,
0xcb,0x2b,0xb7,0xce,0x08,0xa1,0x76,0x1d,0x1a,0xb8,0xe3,0xcc,0x7e,0x48,0x20,0xe6,
0xf8,0x45,0x93,0xde,0xc3,0x63,0x0f,0xb0,0xac,0x5c,0xba,0xdf,0x07,0x77,0xe7,0x4e,
0x1f,0x28,0x10,0x6c,0x59,0xd3,0xdd,0x2d,0x65,0x39,0xb2,0x74,0x84,0x3d,0xf4,0xbd,
0xc7,0x79,0x60,0x0b,0x4d,0x33,0x36,0x25,0xbc,0xe0,0x09,0xcf,0x5b,0xe2,0x38,0x9e,
0xc0,0xef,0xd2,0x16,0x05,0xbe,0x53,0xf7,0xc2,0xc6,0xa2,0x24,0x98,0x1c,0xad,0x04]

diff = [0xf26cb481,0x16a5dc92,0x3c5ba924,0x79b65248,0x2fc64b18,0x615acd29,0xc3b59a42,0x976b2584,
0x6cf281b4,0xa51692dc,0x5b3c24a9,0xb6794852,0xc62f184b,0x5a6129cd,0xb5c3429a,0x6b978425,
0xb481f26c,0xdc9216a5,0xa9243c5b,0x524879b6,0x4b182fc6,0xcd29615a,0x9a42c3b5,0x2584976b,
0x81b46cf2,0x92dca516,0x24a95b3c,0x4852b679,0x184bc62f,0x29cd5a61,0x429ab5c3,0x84256b97]

In [4]:
def _reverse_int(i: int, size=32) -> int:
    '''
    Bitwise reverse an unsigned integer with fixed word size. 
    Not strictly necessary, but like that the least
    significant bit is in the first column.
    '''
    idx = 0
    res = 0
    while i:
        if i & 1:
            res |= 1 << (size-1-idx)
        idx += 1
        i >>= 1
    return res


def _msb(i: int) -> (int, int):
    '''
    Returns the most significant set bit of an unsinged integer 
    and the total number of bits set.
    '''
    if i == 0:
        return (0, 0)
    idx = 0
    count = 0
    while i > 1:
        if i & 1:
            count += 1
        idx += 1
        i >>= 1
    count += 1
    return (1 << idx, count)

def _gf2_get_column(rows: list[int], column: int, n=32) -> int:
    '''
    Helper used to calculcate matrix products.
    Gets a column from a bit-matrix that is encoded as rows of integers
    '''
    res = 0
    for i, row in enumerate(rows):
        res |= ((row >> (n-1-column)) & 1) << (n-1-i)
    return res

def _gf2_sum(i: int) -> int:
    '''
    Helper used to calculcate matrix products.
    Sums entries of a bit-vector in GF(2)^n represented as an int.
    '''
    return _msb(i)[1] % 2